# Setup

In [1]:
import numpy as np
import os
import sklearn.metrics

import models
import util

SEED = 2021
np.random.seed(SEED)

# Stay in top-level directory for consistency
if '/src' in os.getcwd():
    os.chdir('..')

In [2]:
# Load data
xtrain, ytrain, xtest, ytest, xval, yval = util.load_preg_data(sim=True, onehots=True)
# Create binary labels
ytrain_early, ytrain_late, ytrain_preterm = util.preg_outcome_to_binaries(ytrain)
ytest_early, ytest_late, ytest_preterm = util.preg_outcome_to_binaries(ytest)
yval_early, yval_late, yval_preterm = util.preg_outcome_to_binaries(yval)

# Logistic Regression

## Train

In [11]:
lr_preterm = models.build_logreg()
lr_preterm.fit(xtrain, ytrain_preterm)

/home/cove/Documents/S2021/cpsc464/fair-child/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1, class_weight='balanced')

In [20]:
# lr_late = models.build_logreg()
# lr_late.fit(xtrain, ytrain_late)

In [21]:
# lr_early = models.build_logreg()
# lr_early.fit(xtest, ytest_early)

## Eval

In [22]:
print(lr_preterm.score(xtest, ytest_preterm))
print(sklearn.metrics.classification_report(ytest_preterm, lr_preterm.predict(xtest), digits=4))

# print(lr_late.score(xtest, ytest_late))
# print(sklearn.metrics.classification_report(ytest_late, lr_late.predict(xtest), digits=4))

# print(lr_early.score(xtest, ytest_early))
# print(sklearn.metrics.classification_report(ytest_early, logreg_preterm.predict(xtest), digits=4))

0.9695571955719557
              precision    recall  f1-score   support

       False     0.9646    0.9737    0.9691       532
        True     0.9744    0.9656    0.9700       552

    accuracy                         0.9696      1084
   macro avg     0.9695    0.9696    0.9696      1084
weighted avg     0.9696    0.9696    0.9696      1084



In [19]:
print(yval_preterm.any())
# print(yval_late.any())
# print(yval_early.any())

True
True
False


So we just don't have enough examples of early stillbirths in the synthetic data. Hopefully real data will ameliorate this.

# Gradient Boosting

In [26]:
gb_preterm = models.build_gbdt()
gb_preterm.fit(xtrain, ytrain_preterm, eval_set=(xval, yval_preterm), eval_metric='auc', verbose=100)

[100]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.610603
[200]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.542087
[300]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.484359
[400]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.435218
[500]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.39304
[600]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.356594
[700]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.324925
[800]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.297276
[900]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.273042
[1000]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.251728
[1100]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.232927
[1200]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.216301
[1300]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.201565
[1400]	valid_0's auc: 0.995474	valid_0's binary_logloss: 0.188479
[1500]	valid_0's auc: 0.998312	valid_0's binary_logloss: 0.175566
[1600]	valid_0's auc

LGBMClassifier(is_unbalance=True, learning_rate=0.001, min_child_samples=500,
               n_estimators=2000, num_leaves=48, objective='binary')